In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [102]:
import random

filename = list(range(0, 1000))
pred = random.choices(range(0, 4), k=1000)
a = ['a' for i in range(500)]
b = ['b' for i in range(500)]
pred2 = a + b

In [ ]:
def my_model_predict(test_generator, MODEL_PATH):
  import numpy as np
  import pandas as pd
  import tensorflow as tf
  from tensorflow import keras

  model = keras.models.load_model()
  # model.summary()
  test_generator.reset()
  pred = model.predict(test_generator)
  pred = np.argmax(pred, axis=1)

  fn = test_generator.filenames
  filename = [os.path.splitext(os.path.basename(x))[0] for x in fn]

  predict_df = pd.DataFrame({
      'filename': filename,
      'y_pred': pred
  })

  return predict_df

In [119]:
def predict_to_table(p_df):
    from datetime import datetime, timedelta
    classes = pd.Series({0: '결혼해두목', 1: '두분사망토론', 2: '사이코러스', 3: '코빅엔터'})
    predict_df = p_df.copy()
    predict_df['y_pred'] = predict_df['y_pred'].map(classes)
    predict_df['filename'] = predict_df['filename'].astype('int')
    predict_df.sort_values(by='filename', ignore_index=True, inplace=True)
    predict_df['filename'] = pd.to_datetime(predict_df['filename'], unit='s')
    predict_df['filename'] = predict_df['filename'].dt.time

    predict_df['next'] = predict_df['y_pred'].shift(1)
    predict_df.loc[0, 'next'] = predict_df.loc[0, 'y_pred']
    predict_df['TF'] = (predict_df['next'] == predict_df['y_pred'])

    df_rows = predict_df.shape[0]

    df = pd.DataFrame(columns=['corner', 'startTime', 'endTime'])
    p = pd.DataFrame({'corner': predict_df.loc[0, 'y_pred'], 'startTime': predict_df.loc[0, 'filename'], 'endTime': ''}, index=[0])

    for row in range(df_rows):
        if row == df_rows - 1:
            if predict_df.loc[row, 'TF']:
                p.loc[0, 'endTime'] = predict_df.loc[row, 'filename']
                df = pd.concat([df, p], axis=0)
            else:
                tmp = predict_df.loc[row, 'filename']
                tmp = datetime(100, 1, 1, tmp.hour, tmp.minute, tmp.second)
                tmp += timedelta(seconds=-1)
                tmp = tmp.time()
                p.loc[0, 'endTime'] = tmp
                df = pd.concat([df, p], axis=0)

                p.loc[0, 'corner'] = predict_df.loc[row, 'y_pred']
                p.loc[0, 'startTime'] = predict_df.loc[row, 'filename']
                p.loc[0, 'endTime'] = predict_df.loc[row, 'filename']
                df = pd.concat([df, p], axis=0)
        
        if predict_df.loc[row, 'TF'] == False:
            tmp = predict_df.loc[row, 'filename']
            tmp = datetime(100, 1, 1, tmp.hour, tmp.minute, tmp.second)
            tmp += timedelta(seconds=-1)
            tmp = tmp.time()
            p.loc[0, 'endTime'] = tmp
            df = pd.concat([df, p], axis=0)

            p.loc[0, 'corner'] = predict_df.loc[row, 'y_pred']
            p.loc[0, 'startTime'] = predict_df.loc[row, 'filename']
    
    df.reset_index(inplace=True, drop=True)
    
    return df

In [ ]:
p.loc[0,'tmp_1'] = datetime(100, 1, 1, p.loc[0,'startTime'].hour, p.loc[0,'startTime'].minute, p.loc[0,'startTime'].second)
p.loc[0,'tmp_2'] = datetime(100, 1, 1, p.loc[0,'endTime'].hour, p.loc[0,'endTime'].minute, p.loc[0,'endTime'].second) 
p.loc[0,'duration'] = p.loc[0,'tmp_2'] - p.loc[0,'tmp_1']
p.loc[0,'duration'] = datetime.strptime(str(p.loc[0,'duration']), "%H:%M:%S")

In [121]:
# classes = pd.Series({0: '결혼해두목', 1: '두분사망토론', 2: '사이코러스', 3: '코빅엔터'})
classes = pd.Series({0: 'a', 1: 'b', 2: 'c', 3: 'd'})
# pred = model.predict()

# pred = np.argmax(pred, axis=1)

# fn = test_generator.filenames
# filename = [os.path.splitext(os.path.basename(x))[0] for x in fn]

t = pd.DataFrame({
    'filename': filename,
    'y_pred': pred
})

In [124]:
t

,filename,y_pred
0,0,1
1,1,0
2,2,1
3,3,3
4,4,2
...,...,...
995,995,2
996,996,2
997,997,1
998,998,1


In [123]:
predict_to_table(t)

,corner,startTime,endTime
0,두분사망토론,00:00:00,00:00:00
1,결혼해두목,00:00:01,00:00:01
2,두분사망토론,00:00:02,00:00:02
3,코빅엔터,00:00:03,00:00:03
4,사이코러스,00:00:04,00:00:04
...,...,...,...
742,결혼해두목,00:16:34,00:16:34
743,사이코러스,00:16:35,00:16:36
744,두분사망토론,00:16:37,00:16:38
745,코빅엔터,00:16:39,00:16:39


In [7]:
predict_df['y_pred'] = predict_df['y_pred'].map(classes)
predict_df['filename'] = predict_df['filename'].astype('int')
predict_df.sort_values(by='filename', ignore_index=True, inplace=True)
predict_df['filename'] = pd.to_datetime(predict_df['filename'], unit='s')
predict_df['filename'] = predict_df['filename'].dt.time

In [77]:
from datetime import datetime, timedelta

a = predict_df.loc[3, 'filename']
a_t = datetime(100, 1, 1, a.hour, a.minute, a.second)
b = predict_df.loc[25, 'filename']
b_t = datetime(100, 1, 1, b.hour, b.minute, b.second)

c = b_t - a_t
d = datetime.strptime(str(c), "%H:%M:%S")
print(d.time())

00:00:22


In [60]:
pdf = predict_df.copy()

In [61]:
pdf['next'] = pdf['y_pred'].shift(1)
pdf.loc[0, 'next'] = pdf.loc[0, 'y_pred']
pdf['TF'] = (pdf['next'] == pdf['y_pred'])
pdf

,filename,y_pred,next,TF
0,00:00:00,a,a,True
1,00:00:01,a,a,True
2,00:00:02,a,a,True
3,00:00:03,a,a,True
4,00:00:04,a,a,True
...,...,...,...,...
995,00:16:35,b,b,True
996,00:16:36,b,b,True
997,00:16:37,b,b,True
998,00:16:38,b,b,True


In [62]:
from datetime import datetime, timedelta

In [63]:
tmp = pdf.loc[5, 'filename']
tmp = datetime(100, 1, 1, tmp.hour, tmp.minute, tmp.second)
tmp += timedelta(seconds=-1)
tmp = tmp.time()
tmp

datetime.time(0, 0, 4)

In [64]:
df_rows = pdf.shape[0]

df = pd.DataFrame(columns=['corner', 'startTime', 'endTime'])
p = pd.DataFrame({'corner': pdf.loc[0, 'y_pred'], 'startTime': pdf.loc[0, 'filename'], 'endTime': ''}, index=[0])

for row in range(df_rows):
    if row == df_rows - 1:
        if pdf.loc[row, 'TF']:
            p.loc[0, 'endTime'] = pdf.loc[row, 'filename']
            df = pd.concat([df, p], axis=0)
        else:
            tmp = pdf.loc[row, 'filename']
            tmp = datetime(100, 1, 1, tmp.hour, tmp.minute, tmp.second)
            tmp += timedelta(seconds=-1)
            tmp = tmp.time()
            p.loc[0, 'endTime'] = tmp
            df = pd.concat([df, p], axis=0)

            p.loc[0, 'corner'] = pdf.loc[row, 'y_pred']
            p.loc[0, 'startTime'] = pdf.loc[row, 'filename']
            p.loc[0, 'endTime'] = pdf.loc[row, 'filename']
            df = pd.concat([df, p], axis=0)
    
    if pdf.loc[row, 'TF'] == False:
        tmp = pdf.loc[row, 'filename']
        tmp = datetime(100, 1, 1, tmp.hour, tmp.minute, tmp.second)
        tmp += timedelta(seconds=-1)
        tmp = tmp.time()
        p.loc[0, 'endTime'] = tmp
        df = pd.concat([df, p], axis=0)

        p.loc[0, 'corner'] = pdf.loc[row, 'y_pred']
        p.loc[0, 'startTime'] = pdf.loc[row, 'filename']

In [57]:
predict_df = pd.DataFrame({
    'filename': filename,
    'y_pred': pred2
})

In [58]:
predict_df['filename'] = predict_df['filename'].astype('int')
predict_df.sort_values(by='filename', ignore_index=True, inplace=True)
predict_df['filename'] = pd.to_datetime(predict_df['filename'], unit='s')
predict_df['filename'] = predict_df['filename'].dt.time

In [79]:
df.reset_index()
df['tmp_1'] = datetime(100, 1, 1, df['startTime'].hour, df['startTime'].minute, df['startTime'].second)
df['tmp_2'] = datetime(100, 1, 1, df['endTime'].hour, df['endTime'].minute, df['endTime'].second) 
df['duration'] = df['tmp_2'] - df['tmp_1']
df['duration'] = datetime.strptime(str(df['duration']), "%H:%M:%S")

AttributeError: 'Series' object has no attribute 'hour'

In [83]:
df['startTime'] = datetime(100, 1, 1, df['startTime'].hour, df['startTime'].minute, df['startTime'].second)

AttributeError: 'Series' object has no attribute 'hour'

In [65]:
# 코너명, 시작시간, 끝나는 시간, 방영 총 시간
predict_df

,filename,y_pred
0,00:00:00,a
1,00:00:01,a
2,00:00:02,a
3,00:00:03,a
4,00:00:04,a
...,...,...
995,00:16:35,b
996,00:16:36,b
997,00:16:37,b
998,00:16:38,b
